In [1]:
# make sure working directory is root of repository
%cd ..
%pwd

/Users/lydiareader/git/dcds-500-project-2


'/Users/lydiareader/git/dcds-500-project-2'

In [2]:
from shopping_environment import ShoppingEnvironment
import pandas as pd

## Setup Environment

In [3]:
# parameters
num_shoes        = 1000

# number of each consumer type
num_average     = 9000
num_abnormal    = 1000
num_special     = 250
num_wealthy     = 1000
num_influencers = 500
num_resellers   = 2500
num_total       = num_average + num_abnormal + num_special + num_wealthy + num_influencers + num_resellers

# initialize environment
env = ShoppingEnvironment(
    mean_desire = 0.5,
    std_desire = 0.2,
    desire_threshold = 0.5,
    average_mean_money = 250,
    average_std_money = 15,
    rich_mean_money = 1000,
    rich_std_money = 250,
    average_prob_loyal = 0.5,
    rich_prob_loyal = 0.8,
    reseller_prob_loyal = 0.9,
    num_shoes = num_shoes,
    price = 250,
    num_average = num_average,
    num_abnormal = num_abnormal,
    num_special = num_special,
    num_wealthy = num_wealthy,
    num_influencers = num_influencers,
    num_resellers = num_resellers
)

In [4]:
def get_utilities(df):
    # buyer and seller utility for each simulation
    got_shoes = df[df['shoes_acquired'] >= 1]
    print("Consumer utility: ", got_shoes['desire'].mean())     # buyer utility is average desire of those who purchased shoes
    print("Seller utility: ", got_shoes['influence'].mean())    # seller utility is average influence of those who purchased shoes 


def group_fairness(df):
    ret_dict = {}
    got_shoes = df[df['shoes_acquired'] >= 1]
    ret_dict['Consumer utility'] = got_shoes['desire'].mean()
    ret_dict['Seller utility'] = got_shoes['influence'].mean()
    amount_con_types = {
        'average': num_average,
        'abnormal': num_abnormal,
        'special': num_special,
        'wealthy': num_wealthy,
        'influencer': num_influencers,
        'reseller': num_resellers
    }
    for con, amount in amount_con_types.items():
        ret_dict[f'{con}_num_got_shoes'] = len(got_shoes[got_shoes['identity'] == con])
        ret_dict[f'{con}_percent_got_shoes'] = len(got_shoes[got_shoes['identity'] == con]) / amount
        ret_dict[f'{con}_shoes_bought'] = df.loc[df['identity'] == con, 'shoes_acquired'].sum()
        ret_dict[f'{con}_shoes_per_person'] = df.loc[df['identity'] == con, 'shoes_acquired'].sum() / amount

    return ret_dict
        

## No Gaming

### Lottery
Every consumer has an equal chance of being selected to purchase shoes

In [20]:
env.restock(1000)
env.run_lottery_no_gaming()  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_no_gaming = pd.Series(group_fairness(df), name='Lottery')
lottery_no_gaming

Consumer utility                  0.416673
Seller utility                    0.148627
average_num_got_shoes           394.000000
average_percent_got_shoes         0.043778
average_shoes_bought            596.000000
average_shoes_per_person          0.066222
abnormal_num_got_shoes           33.000000
abnormal_percent_got_shoes        0.033000
abnormal_shoes_bought            48.000000
abnormal_shoes_per_person         0.048000
special_num_got_shoes             8.000000
special_percent_got_shoes         0.032000
special_shoes_bought             16.000000
special_shoes_per_person          0.064000
wealthy_num_got_shoes            51.000000
wealthy_percent_got_shoes         0.051000
wealthy_shoes_bought            102.000000
wealthy_shoes_per_person          0.102000
influencer_num_got_shoes         27.000000
influencer_percent_got_shoes      0.054000
influencer_shoes_bought          27.000000
influencer_shoes_per_person       0.054000
reseller_num_got_shoes          106.000000
reseller_pe

### First Come First Served
With no gaming, those with highest desire "come first", and so those with the most desire get the sheos

In [6]:
env.restock(num_shoes = 1000)        # reset environment to 100 shoes with no one purchased yet
env.run_first_come_no_gaming()
df = env.get_consumer_df()
first_no_gaming = pd.Series(group_fairness(df), name='First-come')
first_no_gaming

Consumer utility                  0.910674
Seller utility                    0.055210
average_num_got_shoes           509.000000
average_percent_got_shoes         0.056556
average_shoes_bought            787.000000
average_shoes_per_person          0.087444
abnormal_num_got_shoes           47.000000
abnormal_percent_got_shoes        0.047000
abnormal_shoes_bought            64.000000
abnormal_shoes_per_person         0.064000
special_num_got_shoes            16.000000
special_percent_got_shoes         0.064000
special_shoes_bought             32.000000
special_shoes_per_person          0.128000
wealthy_num_got_shoes            46.000000
wealthy_percent_got_shoes         0.046000
wealthy_shoes_bought             92.000000
wealthy_shoes_per_person          0.092000
influencer_num_got_shoes         25.000000
influencer_percent_got_shoes      0.050000
influencer_shoes_bought          25.000000
influencer_shoes_per_person       0.050000
reseller_num_got_shoes            0.000000
reseller_pe

### Invitation Only
All influencers are invited, and the rest of the spots are randomly selected from those with loyalty status

In [7]:
env.restock(num_shoes = 1000)
env.run_invitation_no_gaming()
df = env.get_consumer_df()
invite_no_gaming = pd.Series(group_fairness(df), name='invite_no_gaming')
invite_no_gaming

Consumer utility                  0.458660
Seller utility                    0.348466
average_num_got_shoes           217.000000
average_percent_got_shoes         0.024111
average_shoes_bought            315.000000
average_shoes_per_person          0.035000
abnormal_num_got_shoes           22.000000
abnormal_percent_got_shoes        0.022000
abnormal_shoes_bought            33.000000
abnormal_shoes_per_person         0.033000
special_num_got_shoes             8.000000
special_percent_got_shoes         0.032000
special_shoes_bought             16.000000
special_shoes_per_person          0.064000
wealthy_num_got_shoes            14.000000
wealthy_percent_got_shoes         0.014000
wealthy_shoes_bought             28.000000
wealthy_shoes_per_person          0.028000
influencer_num_got_shoes        500.000000
influencer_percent_got_shoes      1.000000
influencer_shoes_bought         500.000000
influencer_shoes_per_person       1.000000
reseller_num_got_shoes           54.000000
reseller_pe

In [8]:
no_gaming = pd.DataFrame([lottery_no_gaming, first_no_gaming, invite_no_gaming])

## With Gaming

### Lottery
Every consumer has an equal chance of being selected to purchase shoes

In [9]:
env.restock(1000)
env.run_lottery_with_gaming()  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_with_gaming = pd.Series(group_fairness(df), name='lottery_with_gaming')
lottery_with_gaming

Consumer utility                  0.165921
Seller utility                    0.354582
average_num_got_shoes           128.000000
average_percent_got_shoes         0.014222
average_shoes_bought            210.000000
average_shoes_per_person          0.023333
abnormal_num_got_shoes           16.000000
abnormal_percent_got_shoes        0.016000
abnormal_shoes_bought            24.000000
abnormal_shoes_per_person         0.024000
special_num_got_shoes             2.000000
special_percent_got_shoes         0.008000
special_shoes_bought              4.000000
special_shoes_per_person          0.016000
wealthy_num_got_shoes            20.000000
wealthy_percent_got_shoes         0.020000
wealthy_shoes_bought             40.000000
wealthy_shoes_per_person          0.040000
influencer_num_got_shoes          2.000000
influencer_percent_got_shoes      0.004000
influencer_shoes_bought           2.000000
influencer_shoes_per_person       0.004000
reseller_num_got_shoes          334.000000
reseller_pe

### First Come First Served
With gaming, those with highest desire "come first", and so those with the most desire get the sheos

In [10]:
env.restock(num_shoes = 1000)        # reset environment to 100 shoes with no one purchased yet
env.run_first_come_with_gaming()

df = env.get_consumer_df()
first_with_gaming = pd.Series(group_fairness(df), name='first_with_gaming')
first_with_gaming

Consumer utility                  0.825946
Seller utility                    0.170029
average_num_got_shoes           200.000000
average_percent_got_shoes         0.022222
average_shoes_bought            312.000000
average_shoes_per_person          0.034667
abnormal_num_got_shoes           16.000000
abnormal_percent_got_shoes        0.016000
abnormal_shoes_bought            23.000000
abnormal_shoes_per_person         0.023000
special_num_got_shoes            13.000000
special_percent_got_shoes         0.052000
special_shoes_bought             26.000000
special_shoes_per_person          0.104000
wealthy_num_got_shoes            53.000000
wealthy_percent_got_shoes         0.053000
wealthy_shoes_bought            106.000000
wealthy_shoes_per_person          0.106000
influencer_num_got_shoes         13.000000
influencer_percent_got_shoes      0.026000
influencer_shoes_bought          13.000000
influencer_shoes_per_person       0.026000
reseller_num_got_shoes           52.000000
reseller_pe

### Invitation Only
All influencers are invited, and the rest of the spots are randomly selected from those with loyalty status

In [11]:
env.restock(num_shoes = 1000)
env.run_invitation_with_gaming()
df = env.get_consumer_df()
invite_with_gaming = pd.Series(group_fairness(df), name='invite_with_gaming')
invite_with_gaming

Consumer utility                  0.352796
Seller utility                    0.484501
average_num_got_shoes            22.000000
average_percent_got_shoes         0.002444
average_shoes_bought             44.000000
average_shoes_per_person          0.004889
abnormal_num_got_shoes            1.000000
abnormal_percent_got_shoes        0.001000
abnormal_shoes_bought             2.000000
abnormal_shoes_per_person         0.002000
special_num_got_shoes             0.000000
special_percent_got_shoes         0.000000
special_shoes_bought              0.000000
special_shoes_per_person          0.000000
wealthy_num_got_shoes             9.000000
wealthy_percent_got_shoes         0.009000
wealthy_shoes_bought             18.000000
wealthy_shoes_per_person          0.018000
influencer_num_got_shoes        500.000000
influencer_percent_got_shoes      1.000000
influencer_shoes_bought         500.000000
influencer_shoes_per_person       1.000000
reseller_num_got_shoes          210.000000
reseller_pe

## Change cap to 1
Change the cap to one to prevent resellers get too many shoes and ensure the fairness

In [12]:
env.restock(1000)
env.run_lottery_with_gaming(cap=1)  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_cap1_with_gaming = pd.Series(group_fairness(df), name='lottery_cap1_with_gaming')
lottery_cap1_with_gaming

Consumer utility                  0.178305
Seller utility                    0.358671
average_num_got_shoes           224.000000
average_percent_got_shoes         0.024889
average_shoes_bought            227.000000
average_shoes_per_person          0.025222
abnormal_num_got_shoes           27.000000
abnormal_percent_got_shoes        0.027000
abnormal_shoes_bought            27.000000
abnormal_shoes_per_person         0.027000
special_num_got_shoes             0.000000
special_percent_got_shoes         0.000000
special_shoes_bought              0.000000
special_shoes_per_person          0.000000
wealthy_num_got_shoes            52.000000
wealthy_percent_got_shoes         0.052000
wealthy_shoes_bought             52.000000
wealthy_shoes_per_person          0.052000
influencer_num_got_shoes          7.000000
influencer_percent_got_shoes      0.014000
influencer_shoes_bought           7.000000
influencer_shoes_per_person       0.014000
reseller_num_got_shoes          578.000000
reseller_pe

## Authentication

With auth, it's harder for reseller get fake acccounts

In [13]:
env.restock(1000)
env.run_lottery_with_gaming_auth(cap=2)  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_authen_with_gaming = pd.Series(group_fairness(df), name='lottery_authen_with_gaming')
lottery_authen_with_gaming

Consumer utility                  0.171320
Seller utility                    0.348684
average_num_got_shoes           134.000000
average_percent_got_shoes         0.014889
average_shoes_bought            191.000000
average_shoes_per_person          0.021222
abnormal_num_got_shoes           22.000000
abnormal_percent_got_shoes        0.022000
abnormal_shoes_bought            35.000000
abnormal_shoes_per_person         0.035000
special_num_got_shoes             5.000000
special_percent_got_shoes         0.020000
special_shoes_bought             10.000000
special_shoes_per_person          0.040000
wealthy_num_got_shoes            11.000000
wealthy_percent_got_shoes         0.011000
wealthy_shoes_bought             22.000000
wealthy_shoes_per_person          0.022000
influencer_num_got_shoes          6.000000
influencer_percent_got_shoes      0.012000
influencer_shoes_bought           6.000000
influencer_shoes_per_person       0.012000
reseller_num_got_shoes          354.000000
reseller_pe

In [14]:
env.restock(1000)
env.run_lottery_with_gaming_auth(cap=1)  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_authen_cap1_with_gaming = pd.Series(group_fairness(df), name='lottery_authen_cap1_with_gaming')
lottery_authen_cap1_with_gaming

Consumer utility                  0.180541
Seller utility                    0.337310
average_num_got_shoes           265.000000
average_percent_got_shoes         0.029444
average_shoes_bought            265.000000
average_shoes_per_person          0.029444
abnormal_num_got_shoes           35.000000
abnormal_percent_got_shoes        0.035000
abnormal_shoes_bought            35.000000
abnormal_shoes_per_person         0.035000
special_num_got_shoes             0.000000
special_percent_got_shoes         0.000000
special_shoes_bought              0.000000
special_shoes_per_person          0.000000
wealthy_num_got_shoes            31.000000
wealthy_percent_got_shoes         0.031000
wealthy_shoes_bought             31.000000
wealthy_shoes_per_person          0.031000
influencer_num_got_shoes          9.000000
influencer_percent_got_shoes      0.018000
influencer_shoes_bought           9.000000
influencer_shoes_per_person       0.018000
reseller_num_got_shoes          582.000000
reseller_pe

## Combine three distribution methods

Since it's easier for resellers to use bot online, shoes will be distributed in a proportion that fewer percentage of them can be bought online to prevent that

In [15]:
env.restock(200)
env.run_lottery_with_gaming_auth()
env.restock_without_reset(500)
env.run_first_come_with_gaming()
env.restock_without_reset(300)
env.run_invitation_with_gaming()
df = env.get_consumer_df()
combined_distribution_with_gaming = pd.Series(group_fairness(df), name='combined_distribution_with_gaming')
combined_distribution_with_gaming

Consumer utility                  0.540012
Seller utility                    0.373711
average_num_got_shoes           125.000000
average_percent_got_shoes         0.013889
average_shoes_bought            193.000000
average_shoes_per_person          0.021444
abnormal_num_got_shoes           14.000000
abnormal_percent_got_shoes        0.014000
abnormal_shoes_bought            18.000000
abnormal_shoes_per_person         0.018000
special_num_got_shoes             8.000000
special_percent_got_shoes         0.032000
special_shoes_bought             16.000000
special_shoes_per_person          0.064000
wealthy_num_got_shoes            29.000000
wealthy_percent_got_shoes         0.029000
wealthy_shoes_bought             58.000000
wealthy_shoes_per_person          0.058000
influencer_num_got_shoes        308.000000
influencer_percent_got_shoes      0.616000
influencer_shoes_bought         308.000000
influencer_shoes_per_person       0.616000
reseller_num_got_shoes           98.000000
reseller_pe

## Get Final Results

In [16]:
final_result = pd.DataFrame([lottery_no_gaming, first_no_gaming, invite_no_gaming,lottery_with_gaming,first_with_gaming,invite_with_gaming,lottery_cap1_with_gaming,lottery_authen_with_gaming,lottery_authen_cap1_with_gaming,combined_distribution_with_gaming])

In [17]:
# uncomment to save results
# final_result.to_csv('project2_results.txt',sep = '\t') 
final_result

,Consumer utility,Seller utility,average_num_got_shoes,average_percent_got_shoes,average_shoes_bought,average_shoes_per_person,abnormal_num_got_shoes,abnormal_percent_got_shoes,abnormal_shoes_bought,abnormal_shoes_per_person,...,wealthy_shoes_bought,wealthy_shoes_per_person,influencer_num_got_shoes,influencer_percent_got_shoes,influencer_shoes_bought,influencer_shoes_per_person,reseller_num_got_shoes,reseller_percent_got_shoes,reseller_shoes_bought,reseller_shoes_per_person
lottery_no_gaming,0.415997,0.119548,419.0,0.046556,639.0,0.071000,37.0,0.037,53.0,0.053,...,72.0,0.072,18.0,0.036,18.0,0.036,94.0,0.0376,188.0,0.0752
first_no_gaming,0.910674,0.055210,509.0,0.056556,787.0,0.087444,47.0,0.047,64.0,0.064,...,92.0,0.092,25.0,0.050,25.0,0.050,0.0,0.0000,0.0,0.0000
invite_no_gaming,0.458660,0.348466,217.0,0.024111,315.0,0.035000,22.0,0.022,33.0,0.033,...,28.0,0.028,500.0,1.000,500.0,1.000,54.0,0.0216,108.0,0.0432
lottery_with_gaming,0.165921,0.354582,128.0,0.014222,210.0,0.023333,16.0,0.016,24.0,0.024,...,40.0,0.040,2.0,0.004,2.0,0.004,334.0,0.1336,720.0,0.2880
first_with_gaming,0.825946,0.170029,200.0,0.022222,312.0,0.034667,16.0,0.016,23.0,0.023,...,106.0,0.106,13.0,0.026,13.0,0.026,52.0,0.0208,520.0,0.2080
invite_with_gaming,0.352796,0.484501,22.0,0.002444,44.0,0.004889,1.0,0.001,2.0,0.002,...,18.0,0.018,500.0,1.000,500.0,1.000,210.0,0.0840,436.0,0.1744
lottery_cap1_with_gaming,0.178305,0.358671,224.0,0.024889,227.0,0.025222,27.0,0.027,27.0,0.027,...,52.0,0.052,7.0,0.014,7.0,0.014,578.0,0.2312,671.0,0.2684
lottery_authen_with_gaming,0.171320,0.348684,134.0,0.014889,191.0,0.021222,22.0,0.022,35.0,0.035,...,22.0,0.022,6.0,0.012,6.0,0.012,354.0,0.1416,736.0,0.2944
lottery_authen_cap1_with_gaming,0.180541,0.337310,265.0,0.029444,265.0,0.029444,35.0,0.035,35.0,0.035,...,31.0,0.031,9.0,0.018,9.0,0.018,582.0,0.2328,653.0,0.2612
combined_distribution_with_gaming,0.540012,0.373711,125.0,0.013889,193.0,0.021444,14.0,0.014,18.0,0.018,...,58.0,0.058,308.0,0.616,308.0,0.616,98.0,0.0392,407.0,0.1628


In [18]:
final_result.to_csv('project-2-results-updated.csv')